# Prescriptive Analytics Demo: Prioritizing Preventive Care Outreach
This demo identifies patients over 50 who have not had a recent preventive care visit and prioritizes them for outreach based on healthcare expenses.

**Data Source**: Synthea Coherent Synthetic EHR  
**Tools**: SQLite + SQL in Python (via pandas and sqlite3)

In [ ]:
import pandas as pd
import sqlite3

# Load CSV files
patients = pd.read_csv('patients.csv')
encounters = pd.read_csv('coherent-encounter200.csv')

# Create SQLite database
conn = sqlite3.connect(':memory:')
patients.to_sql('patients', conn, index=False, if_exists='replace')
encounters.to_sql('encounters', conn, index=False, if_exists='replace')

### Step 1: Filter patients over age 50

In [ ]:
query_1 = '''
SELECT Id AS patient_id, 
       BIRTHDATE, 
       CAST((julianday('now') - julianday(BIRTHDATE))/365.25 AS INT) AS age,
       HEALTHCARE_EXPENSES
FROM patients
WHERE age >= 50
'''
patients_over_50 = pd.read_sql_query(query_1, conn)
patients_over_50.head()

### Step 2: Identify last preventive visit (if any)

In [ ]:
query_2 = '''
SELECT PATIENT AS patient_id, 
       MAX(START) AS last_preventive
FROM encounters
WHERE DESCRIPTION LIKE '%wellness%'
GROUP BY PATIENT
'''
preventive_visits = pd.read_sql_query(query_2, conn)
preventive_visits.head()

### Step 3: Join and filter patients with no recent preventive visit

In [ ]:
merged = patients_over_50.merge(preventive_visits, on='patient_id', how='left')
merged['last_preventive'] = pd.to_datetime(merged['last_preventive'], errors='coerce')
cutoff = pd.Timestamp.today() - pd.DateOffset(months=12)
merged_filtered = merged[(merged['last_preventive'].isna()) | (merged['last_preventive'] < cutoff)]
merged_filtered_sorted = merged_filtered.sort_values(by='HEALTHCARE_EXPENSES', ascending=False)
merged_filtered_sorted[['patient_id', 'age', 'HEALTHCARE_EXPENSES', 'last_preventive']].head(10)

**Interpretation**: These patients are aged 50+, have no recent preventive visit, and are sorted by healthcare expenses—suggesting higher cost/risk. They should be prioritized for outreach.

You can modify this logic to include location, gender, or known chronic conditions from other Synthea tables.